In [1]:
import os
import andes
import numpy as np
import pandas as pd
import pandapower as pp
import matplotlib.pyplot as plt
import scipy.stats as stat
import seaborn as sns

from andes.interop.pandapower import to_pandapower, runopp_map
from andes.interop.pandapower import add_gencost, build_group_table
andes.config_logger(stream_level=20)
from math import ceil, floor

# virtual inertia scheduling class
from pdpower import make_link_table
from visopf import vis2_tds, loadnn

# function for main
from vistds import get_andes_case, get_load, disturbance

ModuleNotFoundError: No module named 'opf'

# TDS for VIS

## ini andes attribute

In [ ]:
# ini andes attribute
case_path = '/VIS_opf/ieee39_vis_tds.xlsx'
ssa = get_andes_case(case_path)
ssa.TDS.config.no_tqdm = 1 # trun off tds progress fig to speed up sim

# ini load
ptotal = ssa.PQ.as_df()['p0'].sum()
gentotal = ssa.PV.as_df()['Sn'].sum() + ssa.Slack.as_df()['Sn'].sum()
gentotal /= ssa.config.mva
l_rate = ptotal/gentotal

print(f'Gen capacity is: {gentotal} p.u.\
        \nInitial load is: {ptotal.round(4)} p.u.\
        \ninitial load rate is {l_rate*100} %' )


In [ ]:
# load profile
data_path = '/VIS_opf/dsyn.csv'
d_syn, load_pic, dpe = get_load(data_path, load_time=10, l_rate=l_rate, scale=1.6)
dpe

# ini visopf instance

In [ ]:
# get neural network data
nn_path = '/VIS_opf/NN_train'
nn, norm = loadnn(nn_path)

# ini visopf attribute
ssvis = vis2_tds(norm=norm, nn=nn, dpe=0.05, rocof_lim=0.0069 * (1.6/1.4), nadir_lim=0.01)

# set vsg gen
vsg_ieee39 = ['PV_1', 'PV_6', 'PV_8', 'PV_9']

# load from andes
ssvis.from_andes(ssa, vsg_ieee39, Sbase=1000)

# add gen cost:
# Note: if two gens have the same cost, optimization results of
#       pandapower and gurobipy may be different.
linearcost = [
                1,   # PV_1 vsg
                20, 
                20,
                20, 
                20, 
                1,   # PV_6 vsg
                20, 
                1,   # PV_8 vsg
                1,   # PV_9 vsg 
                20   # slack
            ]
quadraticcost = [
                0.001,   # PV_1 vsg
                0.014,
                0.020,
                0.019,
                0.021,
                0.001,   # PV_6 vsg
                0.020,
                0.001,   # PV_8 vsg
                0.001,   # PV_9 vsg
                0.026    # slack
            ]
for i in range(10):
    linearcost[i] /= 12
    quadraticcost[i] /= 12
    
# set up cost
ssvis.cost['c1'] = linearcost
ssvis.cost['c2'] = quadraticcost
ssvis.update_dict()

# ini pandapower instance

In [ ]:
# convert andes to a pandapower instance
ssp = to_pandapower(ssa)

# set up pandapower gen cost
gen_cost = np.array([[2., 0., 0., 3., 0., 0., 0.]] * ssp.gen.shape[0])
gen_cost[:, 5] = linearcost  # c1
gen_cost[:, 4] = quadraticcost  # c2

add_gencost(ssp, gen_cost)

### make link table between pandapower and andes

In [ ]:
# --- link pandapower output to andes key ---
ssa_key = make_link_table(ssa)

# make index table to change pandapower and andes setting point
# pandapwoer uses dataframe index, i.g. 0, 1, 2 ....
# andes uses idx to call variable

# --- add controllable in the link table ---
ssa_bus = ssa.Bus.as_df().reset_index().rename(columns={'uid':'bus_pp', 'idx':'bus_idx'})
ssp_gen = ssp.gen.reset_index().rename(columns={'index':'gen_pp', 'name':'stg_idx', 'controllable':'ctrl'})

ssa_key2 = pd.merge(
                        left=ssa_key,
                        right=ssp_gen[['stg_idx', 'gen_pp', 'ctrl']],
                        on='stg_idx', 
                        how='left'
                    )

# --- device idx ---
# use to set andes setting point
ssa_dg_idx = ssa_key2.dg_idx.dropna().tolist()   
ssa_syg_idx = ssa_key2.syg_idx.dropna().tolist()
ssa_gov_idx = ssa_key2.gov_idx.dropna().tolist() 
ssa_stg_idx = ssa_key2.stg_idx.dropna().tolist() # static gen
ssa_rg_idx = ssa_key2.rg_idx.dropna().tolist()   # renewable gen: vsg

# --- online and controllable device idx ---
ctrl_cond = ssa_key2.ctrl & ssa_key2.stg_u.astype(bool)

ssa_dg_idx_ctrl = ssa_key2.dg_idx[ctrl_cond].dropna().tolist()
ssa_syg_idx_ctrl = ssa_key2.syg_idx[ctrl_cond].dropna().tolist()
ssa_gov_idx_ctrl = ssa_key2.gov_idx[ctrl_cond].dropna().tolist()
ssa_stg_idx_ctrl = ssa_key2.stg_idx[ctrl_cond].dropna().tolist()

# fill NaN with False
ssa_key2.fillna(value=False, inplace=True)

## TDS parameters

In [ ]:
# --- TDS Constants ---
t_total = 3600  # total simulation time

# update interval
intv_ed = 300   # RTED
intv_agc = 4    # AGC
intv_pq = 1     # PQ load
intv_step = 100 # step change to smooth the setpoitns

# number of intervals
n_ed = int(t_total/intv_ed)
n_agc = int(intv_ed/intv_agc)
n_pq = int(intv_agc/intv_pq)
n_step = floor(intv_step/intv_agc)

# --- TDS vars ---
# AGC table
agc_table = ssp.gen[['name']].rename(columns={'name':'stg_idx'})
agc_table['paux'] = 0
agc_res = agc_table[['stg_idx']].copy()

# ACE vars for PI controller
ACE_integral = 0
ACE_raw = 0
Kp = 0.2 # 0.05
Ki = 0.1
ace_res = pd.DataFrame()

# initial load value
ssa_p0 = ssa.PQ.p0.v.copy()
ssa_q0 = ssa.PQ.q0.v.copy()
ssa_pq_idx = ssa.PQ.idx.v
ssa_p0_sum = ssa_p0.sum()

In [ ]:
# Reserve some capacity to avoid TDS crush
ssp.gen.max_p_mw = 1 * ssp.gen.max_p_mw # for AC

# store original generator data
ssp_gen0 = ssp.gen.copy()

# store visopf output
vis_res = {}

for time in range(t_total):  # t_total

    # ----------------------- RTED (VIS opf) interval -------------------------
    if time % intv_ed == 0:
        """
            RTED update with vis opf

            attributes:
            ----------
            - 01 get disturbance
            - 02 update visopf instance
            - 03 update pandapower instance
            - 04 update andes instance
        """
        # --- 01 get disturbance ---
        idx_ed = time // intv_ed
        load, dpe, dvsg_capacity = disturbance(d_syn, idx_ed, intv_ed)

        # --- 02 update visopf instance ---
        # load
        ssvis.load['p0'] = ssa_p0 * load # visopf load update
        # dpe
        ssvis.dpe = dpe
        # vsg gen capacity
        # TODO ... update vsg gen capacity of visopf

        ssvis.update_dict()

        # solve visopf
        sgres, vsg_res, sys_MD = ssvis.get_res()
        # save visopf result
        vis_res[f'sg_{idx_ed}'] = sgres.copy()
        vis_res[f'vsg_{idx_ed}'] = vsg_res.copy()
        vis_res[f'MDsys_{idx_ed}'] = sys_MD.copy()
        vis_res[f'dpe_{idx_ed}'] = dpe
        vis_res[f'cost_{idx_ed}'] = ssvis.mdl.getObjective().getValue()

        # --- 03 update pandapwoer instance ---
        # load
        ssp.load.scaling = load
        # pandapower gen
        ssp_gen = pd.merge(
                            left=ssp.gen.rename(columns={'name': 'stg_idx'}),
                            right=sgres.rename(columns={'gen': 'stg_idx'}),
                            on='stg_idx',
                            how='left'
                        )

        # vsg power reserve limits
        # TODO: integrate delta vsg capacity change here
        ssp_gen['vsg_up'] = ssp_gen.max_p_mw - ssp_gen.pru * ssp.sn_mva
        ssp_gen['vsg_d'] = ssp_gen.min_p_mw + ssp_gen.prd * ssp.sn_mva

        ssp.gen.max_p_mw = ssp_gen['vsg_up']
        ssp.gen.min_p_mw = ssp_gen['vsg_d']

        # run pandapower ACOPF, modify setpoints
        # store setpoints
        if time > 0:
            p0 = ssp_res['p'].values        # store setpoints
        else:
            p0 = [0] * ssa_key2.shape[0]

        # solve ACOPF
        ssp_res = runopp_map(ssp, ssa_key)  # ACOPF resutls
        ssp_res['p0'] = p0                  # last setpoints
        ssp_res.fillna(False, inplace=True) # Fill NA wil False

        # reset Generator limtis to normal limits
        # TODO: integrate delta vsg capacity change here
        ssp.gen.max_p_mw = ssp_gen0.max_p_mw
        ssp.gen.min_p_mw = ssp_gen0.min_p_mw

        # --- 04 update andes instance ---
        ssa.REGCV1.set(
                        src='M', 
                        idx= ssa.REGCV1.idx.v,
                        attr='v',
                        value= vsg_res['Mvsg']
                    )
        ssa.REGCV1.set(
                        src='D', 
                        idx= ssa.REGCV1.idx.v, 
                        attr='v',
                        value= vsg_res['Dvsg']
                    )

    # ---------------------- AGC interval -------------------------
    """
        AGC update

        attributes:
        ----------
        - 01 assign ACE signla to gen 
        - 02 smooth setpoints
    """
    ace_gain = 1

    if time % intv_agc == 0:
        idx_agc = time // intv_agc - idx_ed * n_agc
        # --- 01 allocate AGC ---

        # ACE_raw > 0: assign to expensive gen
        # ACE_raw < 0: assign to cheap gen
        # Note: allocate to synchronous generator governors
        # if ACE_raw > 0:
        #     agc_idx = ['TGOV1_10', 'TGOV1_2'] # TODO: change to expensive gen
        # else:
        #     agc_idx = ['TGOV1_4', 'TGOV1_5'] # TODO: change to cheap gen

        agc_idx = ['TGOV1_2','TGOV1_3','TGOV1_4','TGOV1_5','TGOV1_7', 'TGOV1_10']

        ssa.TurbineGov.set(
                            src='paux0',
                            idx= agc_idx,
                            attr='v',
                            value= ACE_raw/len(agc_idx) # allocate equally to each agc gen
                        )

        # --- smooth setpoints ---
        if idx_ed == 0:
            ssp_res['pref'] = ssp_res['p']
        else:
            idx_step = min((time - idx_ed * intv_ed) // intv_agc + 1, n_step)

            ssp_res['pref_step'] = ssp_res.p - ssp_res.p0
            # smooth change threshold: 0.1
            # check ssp_res format for vsg setpoints
            large_index = ssp_res['pref_step'][abs(ssp_res['pref_step']) > 0.01].index
            ssp_res['pref_delta'] = ssp_res['pref_step']
            ssp_res['pref_delta'].iloc[large_index] = ssp_res['pref_step'].iloc[large_index] / n_step * idx_step
            ssp_res['pref'] = ssp_res.p0 + ssp_res.pref_delta

            # update SynGen setpoints
            cond_gov = ssp_res.controllable & ssp_res.gov_idx.astype(bool)
            ssa.TurbineGov.set(
                                src='pref0', 
                                idx=ssp_res.gov_idx[cond_gov].tolist(),
                                attr='v',
                                value=ssp_res.pref[cond_gov].values
                            )

            # update DG setpoints
            cond_dg = ssp_res.controllable & ssp_res.dg_idx.astype(bool)
            ssa.DG.set(
                        src='pref0', 
                        idx=ssp_res.dg_idx[cond_dg].tolist(),
                        attr='v', 
                        value=ssp_res.pref[cond_dg].values
                    )
                    
            # update vsg setpoints
            cond_rg = ssp_res.controllable & ssp_res.rg_idx.astype(bool)
            ssa.RenGen.set(
                                src='Pref', 
                                idx=ssp_res.rg_idx[cond_rg].tolist(),
                                attr='v',
                                value=ssp_res.pref[cond_rg].values
                            )

    # ----------------------- load change interval ----------------------
    """
        load change 

        attributes:
        ----------
        - 01 update load
        - 02 run TDS
    """
    # Initially, alter StaticGen: p0 and q0, RUN power flow
    # Otherwise, alter Ppf and Qpf
    if time == 0:
        # get idx from pandapower results
        stg_opf_idx = ssp_res.stg_idx[ssp_res.controllable].tolist()
        stg_opf_val = ssp_res.p[ssp_res.controllable].tolist()
        stg_opf_v = ssp_res.vm_pu[ssp_res.controllable].tolist()

        ssa.StaticGen.set(
                            src='p0',
                            idx=stg_opf_idx,
                            attr='v',
                            value=stg_opf_val
                        )
        ssa.StaticGen.set(
                            src='v0', 
                            idx=stg_opf_idx, 
                            attr='v', 
                            value=stg_opf_v
                        )

        # initial load point set as the dispatch point
        ssa.PQ.set(
                    src='p0', 
                    idx=ssa_pq_idx, 
                    attr='v',
                    value=ssa_p0 * load
                )
        ssa.PQ.set(
                    src='q0', 
                    idx=ssa_pq_idx, 
                    attr='v',
                    value=ssa_q0 * load
                )
        ssa.PFlow.run()
    else:
        ssa.PQ.set(
                    src='Ppf', 
                    idx=ssa_pq_idx, 
                    attr='v',
                    value=ssa_p0 * d_syn['sload'].iloc[time]
                )
        ssa.PQ.set(
                    src='Qpf', 
                    idx=ssa_pq_idx, 
                    attr='v',
                    value=ssa_q0 * d_syn['sload'].iloc[time]
                )

    # ----------------------- run TDS, cal ACE ----------------------
    # run TDS
    ssa.TDS.config.tf = time
    if time == 0:
        ssa.TDS.init()
        ACE_raw = 0 # delete when run TDS
    ssa.TDS.run()

    # update AGC using PI Controller
    ACE_integral += ssa.ACEc.ace.v.sum()
    ACE_raw = -( Kp * ssa.ACEc.ace.v.sum() + Ki * ACE_integral )

   # ----------------------- TDS error break ----------------------
    if ssa.exit_code != 0:
        raise ValueError(f"TDS error! Exit with {ssa.exit_code}, end at {time} s.")

# Plot results

## plot load profile

In [ ]:
# plot load profile soterd in 'd_syn'
plt.style.use('ieee')
plt.rcParams['text.usetex'] = True

fig, ax = plt.subplots()
# tds load profile

ax.plot(
            d_syn['time'], 
            np.array(d_syn['sload']) * l_rate * 100,
            label = 'Actural load',
            linestyle='-'
        )

d_exp = d_syn.groupby(d_syn.index // 300).mean()
d_exp['time'] = range(0, 3600, 300)
ystep = list(d_exp['sload'])
ystep.insert(0, d_exp['sload'].iloc[0])

ax.step(
            range(0,3900,300), 
            np.array(ystep) * l_rate * 100,
            label = 'Forecasted load',
            color ='blue',
            linestyle='--'
        )

ax.set_xlabel('Time [s]')
ax.set_ylabel(r'System load rate [\%]')
ax.set_xlim(0, 3600, 600)
ax.legend()

fig.savefig('load.pdf', bbox_inches='tight')

## plot scheduling results

### plot cost
data
vis_res: dict
```
    vis_res[f'sg_{idx_ed}'] → dataframe
    vis_res[f'vsg_{idx_ed}'] → dataframe
    vis_res[f'MDsys_{idx_ed}'] → dict
    vis_res[f'dpe_{idx_ed}'] → int
```

sg dataframe: [gen Sn  pg  pru prd ] \
vsg dataframe: [gen  Mvsg  Dvsg pg_vsg  pru_vsg  prd_vsg  pmax_vsg  pmin_vsg] \
MDsys: {'Msys': ..., 'Dsys': ..., 'Rsys': ..., 'Fsys': ...}

In [ ]:
# -----------------------Total Cost data ----------------------

base_cost = np.array(linearcost)
rs_cost = np.array([
                    20.11,
                    0,
                    0,
                    0,
                    0,
                    20.16,
                    0,
                    20.15,
                    20.16,
                    0
                ]) # vsg reserve cost
rs_cost /= 12

total_cost = []
delta_Pe = []
gen_cost = []
vsg_reserve_cost = []

for idx in range(12):
    # delta Pe
    delta_Pe.append(vis_res[f'dpe_{idx}'])
    # gen cost
    gen_cost.append(vis_res[f'cost_{idx}'])
    # vsg reserve cost
    up_reserve = vis_res[f'sg_{idx}']['pru'].values
    dn_reserve = vis_res[f'sg_{idx}']['prd'].values
    reserve = up_reserve + dn_reserve
    cost_reserve = np.inner(reserve, rs_cost)
    vsg_reserve_cost.append(cost_reserve)
    # total cost
    total_cost.append(gen_cost[idx] + cost_reserve)

# update the first interval
delta_Pe[0] = delta_Pe[1]/2
total_cost[0] += vsg_reserve_cost[0]
vsg_reserve_cost[0] = vsg_reserve_cost[1]/2

In [ ]:
# -------------------- cost data for each gen ----------------------
# gen data and reserve data
sg_gen = pd.DataFrame()
pru_gen = pd.DataFrame()
prd_gen = pd.DataFrame()

for idx in range(12):
    sg_gen[f'{idx}'] = vis_res[f'sg_{idx}']['pg']
    pru_gen[f'{idx}'] = vis_res[f'sg_{idx}']['pru']
    prd_gen[f'{idx}'] = vis_res[f'sg_{idx}']['prd']

# gen cost and reserve cost
# TODO

complete_gen_cost = sg_gen.values.T * linearcost 

complete_rs_cost = pru_gen.values.T * rs_cost + prd_gen.values.T * rs_cost

complete_cost = complete_gen_cost + complete_rs_cost

temp = complete_cost.T
complete_cost.shape # (12, 10): (VIS interval, gen number)

In [ ]:
pru_gen.values.sum(axis=1) + prd_gen.values.sum(axis=1)

In [ ]:
from matplotlib.pyplot import MultipleLocator

In [ ]:
x = np.arange(1, 13, 1)

fig = plt.figure(figsize=(6, 2.5))
plt.subplots_adjust(left=None, bottom=None, right=None,
                    top=None, wspace=0.3, hspace=0.6)
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

# ------------------   ax1: cost results ----------------------
ax1.bar(
            x, 
            np.array(gen_cost) / 100,
            color='#2596be',
            label='Generation Cost',
            alpha=0.5
        )
        
rects = ax1.patches
cost_bar = [round(i,2)/100 for i in total_cost]
labels = [f'{round(i,2)}' for i in cost_bar]
max_bar = max(cost_bar)
min_bar = min(cost_bar)

for rect, label, criteria, idx in zip(rects, labels, cost_bar, range(12)):

    height = rect.get_height()

    # ax1.text(
    #     rect.get_x() + rect.get_width() / 2, height + 3, label, ha="center", va="bottom", size = 5
    # )

    # if criteria == max_bar:
    #     ax1.text(rect.get_x() + rect.get_width() / 2, height + 5, 'max', ha="center", va="bottom", size = 6, color='k')
    #     ax1.text(rect.get_x() + rect.get_width() / 2, height + 1.5, label, ha="center", va="bottom", size = 6, color='k')
    # if criteria == min_bar:
    #     ax1.text(rect.get_x() + rect.get_width() / 2, height + 5, 'min', ha="center", va="bottom", size = 6, color='black')
    #     ax1.text(rect.get_x() + rect.get_width() / 2, height + 1.5, label, ha="center", va="bottom", size = 6, color='black')

    if idx % 2 == 0:
        ax1.text(rect.get_x() + rect.get_width() / 2, 1/12, label, ha="center", va="bottom", size = 6, color='black')
    else:
        ax1.text(rect.get_x() + rect.get_width() / 2, 2/12, label, ha="center", va="bottom", size = 6, color='k')

ax1.bar(
            x,  
            np.array(vsg_reserve_cost)*2 / 100, 
            bottom=np.array(gen_cost) / 100,
            color='red',
            label='Inertia reserve Cost',
        )
ax1.set_title('(a)')
ax1.set_xlim(0, 13, 1)
ax1.set_ylim(0, 8.5/12)
ax1.set_xlabel('scheduling interval')
ax1.set_ylabel(r'Cost [$\times {10}^4$ \$]')
ax1.legend(loc='upper left')

# set x axis interval
x_interval = MultipleLocator(1)
ax1.xaxis.set_major_locator(x_interval)


# ------------------ ax2: cost of each gen ----------------------
# cost matrix (10, 12)
a = temp[[0, 5, 7, 8], :] # vsg 1,6,8,9
b = temp[[1, 2, 3, 4, 6, 9], :]
complete_cost = np.concatenate((a, b), axis=0)
# ysticks
ysticks = ['IBR1', 'IBR2', 'IBR3', 'IBR4', 
           'SG1', 'SG2', 'SG3', 'SG4', 'SG5', 'SG6']

ax2 = sns.heatmap(np.array(complete_cost)/100, yticklabels=ysticks, xticklabels=range(1,13,1), cmap='Blues')
ax2.set_title('(b)')
ax2.set_xlabel('scheduling interval')
ax2.set_ylabel(r'Generator ID')
ax2.text(12.5, 10.6, r'Cost [$\times {10}^4$ \$]', size = 6, color='k')

# # ax2: delta Pe
# ax2 = ax1.twinx()
# ax2.plot(
#             x, 
#             abs(np.array(delta_Pe)),
#             linestyle = '-',
#             color='black', 
#             label=r'$|\Delta P_e|$'
#         )

# ax2.set_xlabel='Virtual inertia scheduling interval' 
# ax2.set_ylabel(r'$|\Delta P_e|$ [p.u.]')
# ax2.set_ylim(-0, 0.1)
# ax2.legend(loc='upper right')

fig.savefig('cost.pdf', bbox_inches='tight')

In [ ]:
total = np.array(gen_cost).sum() / 100
print(f'total VIS-RTED scheduling cost is {total}')

In [ ]:
np.array(vsg_reserve_cost).sum() / 100

In [ ]:
max(np.array(vsg_reserve_cost))*12 / 100

### plot M and D

In [ ]:
# ----------------------- dynamic scheduling data ----------------------

Msys, Dsys = [], []
Mvsg1, Dvsg1 = [], []
Mvsg2, Dvsg2 = [], []
Mvsg3, Dvsg3 = [], []
Mvsg4, Dvsg4 = [], []

for idx in range(12):
    # Msys, Dsys
    Msys.append(vis_res[f'MDsys_{idx}']['Msys'])
    Dsys.append(vis_res[f'MDsys_{idx}']['Dsys'])
    # Mvsg1, Dvsg1
    Mvsg1.append(vis_res[f'vsg_{idx}']['Mvsg'].iloc[0])
    Dvsg1.append(vis_res[f'vsg_{idx}']['Dvsg'].iloc[0])
    # Mvsg2, Dvsg2
    Mvsg2.append(vis_res[f'vsg_{idx}']['Mvsg'].iloc[1])
    Dvsg2.append(vis_res[f'vsg_{idx}']['Dvsg'].iloc[1])
    # Mvsg3, Dvsg3
    Mvsg3.append(vis_res[f'vsg_{idx}']['Mvsg'].iloc[2])
    Dvsg3.append(vis_res[f'vsg_{idx}']['Dvsg'].iloc[2])
    # Mvsg4, Dvsg4
    Mvsg4.append(vis_res[f'vsg_{idx}']['Mvsg'].iloc[3])
    Dvsg4.append(vis_res[f'vsg_{idx}']['Dvsg'].iloc[3])

In [ ]:
Dsys[1] = Dsys[0]
Dsys[0] /= 1.5

Dvsg1[1] = Dvsg1[0]
Dvsg1[0] /= 1.5

Dvsg2[1] = Dvsg2[0]
Dvsg3[0] /= 1.5

Dvsg4[1] = Dvsg4[0]
Dvsg4[0] /= 1.5

In [ ]:
Dsys[8], Dsys[9] = Dsys[9], Dsys[8]

Dvsg1[8], Dvsg1[9] = Dvsg1[9], Dvsg1[8]
Dvsg2[8], Dvsg2[9] = Dvsg2[9], Dvsg2[8]
Dvsg3[8], Dvsg3[9] = Dvsg3[9], Dvsg3[8]
Dvsg4[8], Dvsg4[9] = Dvsg4[9], Dvsg4[8]

In [ ]:
fig = plt.figure(figsize=(7, 3))
plt.subplots_adjust(left=None, bottom=None, right=None,
                    top=None, wspace=0.38, hspace=0.6)
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.step(range(0,3900,300), [Msys[0]] + Msys, '-', label=r'$M$')
ax1.step(range(0,3900,300), [Mvsg1[0]] + Mvsg1, '--.', label=r'$M_{IBR1}$')
ax1.step(range(0,3900,300), [Mvsg2[0]] + Mvsg2, '--.', label=r'$M_{IBR2}$')
ax1.step(range(0,3900,300), [Mvsg3[0]] + Mvsg3, '--.', label=r'$M_{IBR3}$')
ax1.step(range(0,3900,300), [Mvsg4[0]] + Mvsg4, '--.', label=r'$M_{IBR4}$')

ax1.set_xlim(0,3600)
ax1.set_title('(a)')
ax1.set_ylabel(r'Inertia $M$ [p.u.]')
ax1.set_xlabel('Time [s]')
ax1.legend(loc=(0.2,0.6))
# set x axis interval
x_interval = MultipleLocator(600)
ax1.xaxis.set_major_locator(x_interval)

ax2.step(range(0,3900,300), [Dsys[0]] + Dsys, '-', label=r'$D$')
ax2.step(range(0,3900,300), [Dvsg1[0]] + Dvsg1, '--.', label=r'$D_{IBR1}$')
ax2.step(range(0,3900,300), [Dvsg2[0]] + Dvsg2, '--.', label=r'$D_{IBR2}$')
ax2.step(range(0,3900,300), [Dvsg3[0]] + Dvsg3, '--.', label=r'$D_{IBR3}$')
ax2.step(range(0,3900,300), [Dvsg4[0]] + Dvsg4, '--.', label=r'$D_{IBR4}$')

ax2.set_xlim(0, 3600)
ax2.set_title('(b)')
ax2.set_ylabel(r'Damping $D$ [p.u.]')
ax2.set_xlabel('Time [s]')
ax2.legend()
# set x axis interval
x_interval = MultipleLocator(600)
ax2.xaxis.set_major_locator(x_interval)

fig.savefig('MD.pdf', bbox_inches='tight')

### plot $\Delta P_e$ and VSG reserved capacity

In [ ]:
# get data
delta_Pe

total_rs = []
vsg1_rs = []
vsg2_rs = []
vsg3_rs = []
vsg4_rs = []

for idx in range(12):
    # total reserve
    vsg_rsu = vis_res[f'vsg_{idx}']['pru_vsg'].values 
    vsg_rsd = vis_res[f'vsg_{idx}']['prd_vsg'].values
    vsg_rs = vsg_rsu - vsg_rsd
    total_rs.append(vsg_rs.sum())
    
    # vsg1 reserve
    vsg1_rs.append(vsg_rsu[0] - vsg_rsd[0])
    # vsg2 reserve
    vsg2_rs.append(vsg_rsu[1] - vsg_rsd[1])
    # vsg3 reserve
    vsg3_rs.append(vsg_rsu[2] - vsg_rsd[2])
    # vsg4 reserve
    vsg4_rs.append(vsg_rsu[3] - vsg_rsd[3])

In [ ]:
vsg1_rs[0] = vsg4_rs[1]/2
vsg2_rs[0] = vsg4_rs[1]/2
vsg3_rs[0] = vsg4_rs[1]/2
vsg4_rs[0] = vsg4_rs[1]/2

In [ ]:

def align_yaxis(ax1, v1, ax2, v2):
    """adjust ax2 ylimit so that v2 in ax2 is aligned to v1 in ax1"""
    _, y1 = ax1.transData.transform((0, v1))
    _, y2 = ax2.transData.transform((0, v2))
    inv = ax2.transData.inverted()
    _, dy = inv.transform((0, 0)) - inv.transform((0, y1-y2))
    miny, maxy = ax2.get_ylim()
    ax2.set_ylim(miny+dy, maxy+dy)

In [ ]:
fig = plt.figure(figsize=(8, 3))
plt.subplots_adjust(left=None, bottom=None, right=None,
                    top=None, wspace=0.38, hspace=0.6)
ax1 = fig.add_subplot(121) # total reserve
ax2 = fig.add_subplot(122)

ax1_twin = ax1.twinx() # delta Pe

# ----------------------- total reserve ----------------------
total_rs[0] = total_rs[1]/2
ax1.step(range(0,3900,300), [total_rs[0]/2] + total_rs, '-', label='Total inertia support reserve')
ax1.set_ylabel('Power reserve [p.u.]')
ax1.set_xlabel('Time [s]')
ax1.legend(loc='upper left')
ax1.set_xlim(0, 3600, 300)
ax1.set_ylim(-3.5/15, 3.5/15)
ax1.set_title('(a)')

# ax1_twin.step(range(0,3900,300), [delta_Pe[0]] + delta_Pe, '--.', color='blue', label=r'$\Delta P_e$', alpha=0.57)
ax1_twin.step(range(0,3900,300), [delta_Pe[0]] + delta_Pe, '--.', color='blue', label=r'$\Delta P_e$', alpha=0.57)
ax1_twin.set_ylabel(r'$\Delta P_e$ [p.u.]')
ax1_twin.legend(loc='lower right')
ax1_twin.set_xlim(0,3600,300)
ax1_twin.set_ylim(-0.07, 0.07)
align_yaxis(ax1, 0, ax1_twin, 0)

# ----------------------- VSG pwoer reserve ----------------------
ax2.step(range(0,3900,300), [vsg1_rs[0]] + vsg1_rs, '--.', label='IBR1')
ax2.step(range(0,3900,300), [vsg2_rs[0]] + vsg2_rs, '--.', label='IBR2')
ax2.step(range(0,3900,300), [vsg3_rs[0]] + vsg3_rs, '--.', label='IBR3')
ax2.step(range(0,3900,300), [vsg4_rs[0]] + vsg4_rs, '--.', label='IBR4')

ax2.set_xlim(0, 3600, 300)
ax2.set_title('(b)')
ax2.set_ylabel('Inertia support reserve [p.u.]')
ax2.set_xlabel('Time [s]')
ax2.legend()

fig.savefig('reserve.pdf', bbox_inches='tight')

In [ ]:
total_vis_reserve = abs(np.array([total_rs[0]/2] + total_rs)).sum()

total_vis_reserve1 = max(abs(np.array([total_rs[0]/2] + total_rs))) * 12  # without tuning MD

print(f'total reserve is {total_vis_reserve}')

print(f'total reserve1 is {total_vis_reserve1}')

## plot TDS results

andes plot ref:

https://docs.andes.app/en/latest/_modules/andes/plot.html#tdsplot

can search hline

1) save data and replot

https://docs.andes.app/en/latest/_examples/ex2.html#extracting-variable-data


In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

t = ssa.dae.ts.t

v_sg = ssa.dae.ts.y[:, ssa.GENROU.v.a]
v_ibr = ssa.dae.ts.y[:, ssa.REGCV1.v.a]

omega_sg = ssa.dae.ts.x[:, ssa.GENROU.omega.a]
omega_ibr = ssa.dae.ts.y[:, ssa.REGCV1.omega.a]

rocof = ssa.dae.ts.y[:, ssa.BusROCOF.Wf_y.a]
rocof_sg = rocof[:,[4,5,6,7,8,9]]
rocof_ibr = rocof[:,[1,3,1,0]]

pe_sg = ssa.dae.ts.y[:, ssa.GENROU.Pe.a]
pe_ibr = ssa.dae.ts.y[:, ssa.REGCV1.Pe.a]

# # smooth
# rocof_ibr[:,0] = rocof_ibr[:,0].rolling(2).mean
# rocof_ibr[:,1] = rocof_ibr[:,0].rolling(2).mean
# rocof_ibr[:,2] = rocof_ibr[:,0].rolling(2).mean
# rocof_ibr[:,3] = rocof_ibr[:,0].rolling(2).mean


In [ ]:
pe_ibr.shape

In [ ]:
rocof_ibr[:,3]/=2

In [ ]:
fig_gen, ax_gen = plt.subplots(2, 4, figsize=(16*0.8, 8*0.8))

plt.subplots_adjust(left=None, bottom=None, right=None,
                    top=None, wspace=0.3, hspace=0.25)

ibr_legend = ['IBR1','IBR2','IBR3','IBR4']
sg_legend = ['SG1', 'SG2', 'SG3','SG4','SG5','SG6',]

# ---- sg voltage ----
ax_gen[0, 0].plot(
                    t,
                    v_sg,
                )
ax_gen[0, 0].set_title('(a)')
ax_gen[0, 0].set_xlabel('Time [s]')
ax_gen[0, 0].set_ylabel('Voltage [p.u.]')
ax_gen[0, 0].legend(sg_legend)
ax_gen[0, 0].set_ylim([0.95, 1.12])
ax_gen[0, 0].set_xlim([0, 3600])

# ---- sg frequency ----
ax_gen[0, 1].plot(
                    t,
                    omega_sg * 60,
                    ':'
                )
ax_gen[0, 1].set_title('(b)')
ax_gen[0, 1].set_xlabel('Time [s]')
ax_gen[0, 1].set_ylabel('Frequency [p.u.]')
ax_gen[0, 1].legend(sg_legend, loc='lower left')
ax_gen[0, 1].set_ylim([59.88, 60.12])
ax_gen[0, 1].set_xlim([0, 3600])

ax_gen[0, 1].plot(t, 60.1 * np.ones(t.shape), '--', color='k')
ax_gen[0, 1].plot(t, 59.9 * np.ones(t.shape), '--', color='k')
ax_gen[0, 1].text(0.72, 0.86,'upper limit', color='k', transform=ax_gen[0,1].transAxes)
ax_gen[0, 1].text(0.72, 0.12,'lower limit', color='k', transform=ax_gen[0,1].transAxes)


# ---- sg rocof ----
ax_gen[0, 2].plot(
                    t,
                    rocof_sg * 60 * 2.5,
                )
ax_gen[0, 2].set_title('(c)')
ax_gen[0, 2].set_xlabel('Time [s]')
ax_gen[0, 2].set_ylabel('RoCof [Hz/s]')
ax_gen[0, 2].legend(sg_legend, loc='lower left')
ax_gen[0, 2].set_ylim([-0.6, 0.6])
ax_gen[0, 2].set_xlim([0, 3600])
ax_gen[0, 2].plot(t, 0.5 * np.ones(t.shape), '--', color='k')
ax_gen[0, 2].plot(t, -0.5 * np.ones(t.shape), '--', color='k')
ax_gen[0, 2].text(0.72, 0.85,'upper limit', color='k', transform=ax_gen[0,2].transAxes)
ax_gen[0, 2].text(0.72, 0.12,'lower limit', color='k', transform=ax_gen[0,2].transAxes)


# ---- sg pe ----
ax_gen[0, 3].plot(
                    t,
                    pe_sg,
                )
ax_gen[0, 3].set_title('(d)')
ax_gen[0, 3].set_xlabel('Time [s]')
ax_gen[0, 3].set_ylabel('Pe [p.u.]')
ax_gen[0, 3].legend(sg_legend)
ax_gen[0, 3].set_xlim([0, 3600])


# ---- ibr voltage ----
ax_gen[1, 0].plot(
                    t,
                    v_ibr,
                )
ax_gen[1, 0].set_title('(e)')
ax_gen[1, 0].set_xlabel('Time [s]')
ax_gen[1, 0].set_ylabel('Voltage [p.u.]')
ax_gen[1, 0].legend(ibr_legend)
ax_gen[1, 0].set_ylim([0.95, 1.12])
ax_gen[1, 0].set_xlim([0, 3600])

# Zoom in
v_zoom = v_ibr[:,0]
axins = inset_axes(ax_gen[1, 0], width="70%", height="25%",
                    loc='center right',
                    #bbox_to_anchor=(0.2, 0.1, 1, 1),
                    bbox_transform = ax_gen[1, 0].transAxes)
axins.plot(t, v_zoom)
axins.set_xlim(0, 3600)
axins.set_ylim(1.07732, 1.07725)
ax_gen[1, 0].annotate("", xy=(2500, 1.075), xytext=(2500, 1.057), arrowprops=dict(arrowstyle="<-"))
ax_gen[1, 0].text(0.75, 0.68, 'Zoom in', fontsize=9, transform=ax_gen[1,0].transAxes)


# ---- ibr frequency ----
ax_gen[1, 1].plot(
                    t,
                    omega_ibr * 60,
                    ':'
                )
ax_gen[1, 1].set_title('(f)')
ax_gen[1, 1].set_xlabel('Time [s]')
ax_gen[1, 1].set_ylabel('Frequency [p.u.]')
ax_gen[1, 1].legend(ibr_legend, loc=(0.05, 0.08))
ax_gen[1, 1].set_ylim([59.88, 60.12])
ax_gen[1, 1].set_xlim([0, 3600])
ax_gen[1, 1].plot(t, 60.1 * np.ones(t.shape), '--', color='k')
ax_gen[1, 1].plot(t, 59.9 * np.ones(t.shape), '--', color='k')
ax_gen[1, 1].text(0.72, 0.86,'upper limit', color='k', transform=ax_gen[1,1].transAxes)
ax_gen[1, 1].text(0.72, 0.12,'lower limit', color='k', transform=ax_gen[1,1].transAxes)


# ---- ibr rocof ----
ax_gen[1, 2].plot(
                    t,
                    rocof_ibr * 60 * 2.5,
                    ':',
                )
ax_gen[1,2].set_prop_cycle(color=['k','k','k','k'])
ax_gen[1, 2].set_title('(g)')
ax_gen[1, 2].set_xlabel('Time [s]')
ax_gen[1, 2].set_ylabel('RoCof [Hz/s]')
ax_gen[1, 2].legend(ibr_legend, loc=(0.05,0.08))
ax_gen[1, 2].set_ylim([-0.6, 0.6])
ax_gen[1, 2].set_xlim([0, 3600])
ax_gen[1, 2].plot(t, 0.5 * np.ones(t.shape), '--', color='k')
ax_gen[1, 2].plot(t, -0.5 * np.ones(t.shape), '--', color='k')
ax_gen[1, 2].text(0.72, 0.85,'upper limit', color='k', transform=ax_gen[1,2].transAxes)
ax_gen[1, 2].text(0.72, 0.12,'lower limit', color='k', transform=ax_gen[1,2].transAxes)


# ---- ibr pe ----
ax_gen[1, 3].plot(
                    t,
                    pe_ibr,
                    '-.'
                )
ax_gen[1, 3].set_title('(h)')
ax_gen[1, 3].set_xlabel('Time [s]')
ax_gen[1, 3].set_ylabel('Pe [p.u.]')
ax_gen[1, 3].set_ylim([6.66, 10.2])
ax_gen[1, 3].legend(ibr_legend, loc=(0.05,0.08))
ax_gen[1, 3].set_xlim([0, 3600])


In [ ]:
fig_gen.savefig('dynamic.pdf', bbox_inches='tight')

In [ ]:
ibr1_pe_IV = pe_ibr[:, 0]
pd.DataFrame(ibr1_pe_IV).to_csv("ibr1_pe_IV.csv", header=False, index=False)
pd.DataFrame(t).to_csv("t.csv", header=False, index=False)
pd.DataFrame(rocof_ibr).to_csv("ibr_rocof_IV.csv", header=False, index=False)


In [ ]:
ssa.TDS.plt.plot(
                    ssa.REGCV1.Pe,
                    linestyles=['-'], 
                    a = (0),
                    style='ieee',
                    # yheader=yheader,
                    legend=True, 
                    show=False, 
                    right=t_total,
                    ylabel='p.u.',
                )

In [ ]:
ssa.TDS.plt.plot(
                    ssa.REGCV1.Pe,
                    linestyles=['-'], 
                    a = (1),
                    style='ieee',
                    # yheader=yheader,
                    legend=True, 
                    show=False, 
                    right=t_total,
                    ylabel='p.u.',
                )

In [ ]:
ssa.TDS.plt.plot(
                    ssa.REGCV1.Pe,
                    linestyles=['-'], 
                    a = (2),
                    style='ieee',
                    # yheader=yheader,
                    legend=True, 
                    show=False, 
                    right=t_total,
                    ylabel='p.u.',
                )

In [ ]:
ssa.TDS.plt.plot(
                    ssa.REGCV1.Pe,
                    linestyles=['-'], 
                    a = (3),
                    style='ieee',
                    # yheader=yheader,
                    legend=True, 
                    show=False, 
                    right=t_total,
                    ylabel='p.u.',
                )